In [1]:
"""
# Bug Detection and Fixing with LLM

This Jupyter Notebook takes user-input Python code, classifies it as 'buggy' or 'bug-free' using a fine-tuned LLM, and suggests fixes.
"""

"\n# Bug Detection and Fixing with LLM\n\nThis Jupyter Notebook takes user-input Python code, classifies it as 'buggy' or 'bug-free' using a fine-tuned LLM, and suggests fixes.\n"

In [2]:
!pip install torch transformers datasets jupyter pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
from google.colab import files
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import Trainer, TrainingArguments
import torch
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification
torch.cuda.is_available() # true using T4 GPU on colab

True

In [5]:
uploaded = files.upload()

Saving github_scraped_half.csv to github_scraped_half.csv


In [6]:
csv_filename = "github_scraped_half.csv" # got from the scrape data set from github and some syntax and keyword adding for better training after scraping w3schools

In [7]:
# Load the CSV file into a pandas DataFrame
try:
    df = pd.read_csv(csv_filename)
except FileNotFoundError:
    df = pd.DataFrame({"code": [], "label": []})

In [8]:
# for some cleaning
print("Unique labels before cleanup:")
print(df['label'].unique())

# Filter out rows with unwanted labels
allowed_labels = ['bug-free', 'buggy']
df = df[df['label'].isin(allowed_labels)]

# Inspect the unique labels again
print("Unique labels after cleanup:")
print(df['label'].unique())

Unique labels before cleanup:
['buggy' 'bug-free' 'b)`' 'bug-free ' 'buggy '
 'set2<br>print(joined_set)`']
Unique labels after cleanup:
['buggy' 'bug-free']


In [9]:
# Split the dataset into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.1)  # 10% for validation

In [10]:
# Convert the train and validation DataFrames into Hugging Face datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

In [11]:
# Load pre-trained model and tokenizer
MODEL_NAME = "microsoft/codebert-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

# Inspect the dataset columns
print(train_dataset.column_names)
print(val_dataset.column_names)

# Check unique labels in the dataset to update label mapping
print(set(train_dataset['label']))

# Function to tokenize the code
def tokenize_function(examples):
    return tokenizer(examples['code'], padding=True, truncation=True, max_length=512)

# Apply tokenization to both train and validation datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

# Update the label map according to your actual dataset labels
label_map = {'bug-free': 0, 'buggy': 1}

# Apply the label mapping
train_dataset = train_dataset.map(lambda x: {'label': label_map[x['label']]})
val_dataset = val_dataset.map(lambda x: {'label': label_map[x['label']]})

# Now create a DatasetDict with both 'train' and 'validation' splits
dataset = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset
})

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Map:   0%|          | 0/1485 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Map:   0%|          | 0/165 [00:00<?, ? examples/s]

Map:   0%|          | 0/1485 [00:00<?, ? examples/s]

Map:   0%|          | 0/165 [00:00<?, ? examples/s]

In [12]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback
import torch

# 🚀 Compute multiple metrics (accuracy, precision, recall, F1)
def compute_metrics(p):
    predictions, labels = p
    preds = predictions.argmax(axis=1)  # Convert logits to predicted class labels
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    accuracy = accuracy_score(labels, preds)
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

def train_model():
    # 🔧 Adjusted training arguments
    training_args = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=2,  # Keep only the 2 best models
        logging_strategy="steps",
        logging_steps=10,
        learning_rate=5e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=10,
        weight_decay=0.05,
        warmup_steps=500,
        gradient_accumulation_steps=2,
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        greater_is_better=True,
        fp16=True,
        gradient_checkpointing=True
    )


    # ⚡ Trainer setup with metrics and callbacks
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,

    )

    # ✅ Train the model
    trainer.train()
    return trainer

In [13]:
# Train the model
train_model()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-12-627a2fd28082>:43: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: debayanghosh408 (debayanghosh408-brainware-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.677300,0.668561,0.581818,0.338512,0.581818,0.428004
2,0.643300,0.601869,0.636364,0.645596,0.636364,0.638659
3,0.573800,0.524361,0.690909,0.703620,0.690909,0.692990
4,0.492100,0.522041,0.709091,0.706777,0.709091,0.702554
5,0.607200,0.496314,0.703030,0.701417,0.703030,0.694247
6,0.327700,0.698326,0.709091,0.706595,0.709091,0.706887
7,0.281700,0.793393,0.727273,0.729375,0.727273,0.717780
8,0.186900,0.984517,0.727273,0.725724,0.727273,0.721738
9,0.170800,1.295087,0.739394,0.745037,0.739394,0.728853
10,0.060600,1.459970,0.721212,0.721888,0.721212,0.712251


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [14]:
def classify_code(code):
    """Uses the fine-tuned model to classify code as 'buggy' or 'bug-free'."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)  # Move the model to the correct device

    inputs = tokenizer(code, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)  # Move inputs to same device
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_label = torch.argmax(logits, dim=1).item()

    return "bug-free" if predicted_label == 0 else "buggy"

In [34]:
# Example buggy code input
code_input = """
def main(a, b):
 pr a
"""

# Classify the code using the fine-tuned model
status = classify_code(code_input)

if status == "buggy":
    print("Buggy Code Detected")
else:
    print("Bug-free Code")

Buggy Code Detected


Deployment

In [26]:
model.save_pretrained("bug_detector_model")
tokenizer.save_pretrained("bug_detector_model")


('bug_detector_model/tokenizer_config.json',
 'bug_detector_model/special_tokens_map.json',
 'bug_detector_model/vocab.json',
 'bug_detector_model/merges.txt',
 'bug_detector_model/added_tokens.json',
 'bug_detector_model/tokenizer.json')

In [27]:
!pip install huggingface_hub

In [31]:
from huggingface_hub import notebook_login
notebook_login()

In [33]:
from huggingface_hub import HfApi

# Replace with your Hugging Face username
HF_USERNAME = "felixoder"

api = HfApi()
api.create_repo(repo_id=f"{HF_USERNAME}/bug_detector_model", repo_type="model", private=False)  # Set private=True if needed

api.upload_folder(
    folder_path="bug_detector_model",
    repo_id=f"{HF_USERNAME}/bug_detector_model",
    repo_type="model"
)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/felixoder/bug_detector_model/commit/02cb4135ab5819f3158880e6e0556933240b457c', commit_message='Upload folder using huggingface_hub', commit_description='', oid='02cb4135ab5819f3158880e6e0556933240b457c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/felixoder/bug_detector_model', endpoint='https://huggingface.co', repo_type='model', repo_id='felixoder/bug_detector_model'), pr_revision=None, pr_num=None)